In [1]:
using CUDA
using TypedTables
using DataFrames
using Statistics
using CSV

In [21]:
# CUDA

# function operation_matrices_cuda(A)
#     # C = CUDA.zeros(Float64, size(A))
#     # CUDA.@sync C = inv(A)
#     # return C
#     gpu_matrix = CUDA.cu(A)  # Transfer matrix to GPU
#     inverted_gpu_matrix = inv(gpu_matrix)  # Invert matrix on GPU
#     return CUDA.collect(inverted_gpu_matrix)  # Transfer back to CPU
# end

# function simulate_gpu(A_normal)
#     # Convert your Float64 matrices to Float32 and then to CuArray{Float32}
#     A = CUDA.convert(CuArray{Float32}, A_normal)
#     # Mengukur waktu eksekusi
#     cuda_start = CUDA.@elapsed result_gpu = operation_matrices_cuda(A)
#     return cuda_start
# end

# ---

# function operation_matrices_cuda(A)
#     gpu_matrix = CUDA.cu(A)  # Transfer matrix to GPU
#     inverted_gpu_matrix = inv(gpu_matrix)  # Invert matrix on GPU
#     return CUDA.collect(inverted_gpu_matrix)  # Transfer back to CPU
# end

# function simulate_gpu(A_normal)
#     # Convert your matrices to CuArray
#     A = CUDA.cu(A_normal)
#     # Measure execution time
#     cuda_start = CUDA.@elapsed result_gpu = operation_matrices_cuda(A)
#     return cuda_start
# end

function operation_matrices_cuda(gpu_matrix)
    # Sudah menerima CuArray, tidak perlu konversi lagi
    inverted_gpu_matrix = inv(gpu_matrix)  # Invert matrix on GPU
    return inverted_gpu_matrix  # Keep the inverted matrix on GPU if possible
end

function simulate_gpu(A_normal)
    # Mengkonversi dan menjaga matriks di GPU
    A = CUDA.cu(A_normal)  # Convert to CuArray, Float32 if possible
    
    # Optimalisasi: Pertimbangkan konversi A_normal ke Float32 sebelum konversi ke CuArray jika presisi tinggi tidak diperlukan
    # A = CUDA.cu(Float32.(A_normal))
    
    # Measure execution time
    result_gpu, cuda_start = CUDA.@timed operation_matrices_cuda(A)
    
    # Opsi: Jika Anda perlu matriks hasil di CPU, transfer di sini
    # result_cpu = CUDA.collect(result_gpu)
    
    return cuda_start 
end

simulate_gpu (generic function with 1 method)

In [3]:
# CPU

function operation_matrices_normal(A)
    C = zeros(Float32, size(A))
    C = inv(A)
    return C
end

function simulate_cpu(A_normal)
    normal_start = @elapsed result_cpu = operation_matrices_normal(A_normal)
    # println("Waktu eksekusi dengan CPU: $normal_start detik")
    return normal_start
end

simulate_cpu (generic function with 1 method)

In [4]:
function simulate(N::Int64, n_loop::Int64)
    # println("Total size: $(N*N)")
    results_gpu = []
    results_cpu = []
    is_better_gpu = []
    for i in 1:n_loop
        A_normal = rand(Float32, N, N)
        # B_normal = rand(Float32, N, N)
        benchmark_gpu = simulate_gpu(A_normal)
        benchmark_cpu = simulate_cpu(A_normal)
        push!(results_gpu, benchmark_gpu)
        push!(results_cpu, benchmark_cpu)
        push!(is_better_gpu, benchmark_gpu < benchmark_cpu)
    end
    println("Mean CPU: $(mean(results_cpu))")
    println("Mean GPU: $(mean(results_gpu))")
    return DataFrame(CPU=results_cpu, GPU=results_gpu, Better_GPU=is_better_gpu)
end

simulate (generic function with 1 method)

In [24]:
df = simulate(10, 10)

Mean CPU: 8.2195e-6
Mean GPU: 0.00012493090000000002


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,1.7404e-5,0.000178785,false
2,8.485e-6,0.000114816,false
3,7.135e-6,0.000113152,false
4,6.877e-6,0.00011146,false
5,6.48e-6,0.000114034,false
6,5.88e-6,0.000114583,false
7,6.34e-6,0.000112649,false
8,6.331e-6,0.000113051,false
9,7.105e-6,0.000134021,false


In [16]:
df = simulate(10, 10)
CSV.write("./1.csv", df)
df

Mean CPU: 1.4290099999999999e-5
Mean GPU: 0.00019739517


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,2.9804e-5,0.00034928,false
2,1.5107e-5,0.000198944,false
3,1.1465e-5,0.000187936,false
4,1.1371e-5,0.000182656,false
5,1.1165e-5,0.000176352,false
6,9.739e-6,0.000175744,false
7,2.157e-5,0.000167904,false
8,1.0629e-5,0.000173696,false
9,1.1077e-5,0.000178144,false


In [38]:
df = simulate(50, 10)
CSV.write("./2.csv", df)
df

Mean CPU: 0.0001356064
Mean GPU: 0.00039810556


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.000153234,0.000454112,false
2,0.000138019,0.000448256,false
3,0.000138955,0.000399904,false
4,0.000141373,0.000456384,false
5,0.000132432,0.000358336,false
6,0.000139432,0.00038064,false
7,0.000121847,0.000387456,false
8,0.000128773,0.000372832,false
9,0.000127195,0.00036832,false


In [85]:
df = simulate(100, 10)
CSV.write("./3.csv", df)
df

Total size: 10000
Mean CPU: 0.0023815273
Mean GPU: 0.0005711424


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.000240995,0.000602752,false
2,0.000200691,0.000589504,false
3,0.011989,0.000547456,true
4,0.00201261,0.000596192,true
5,0.00024286,0.00054912,false
6,0.000233224,0.000545536,false
7,0.00021233,0.000524576,false
8,0.00820464,0.00052816,true
9,0.000261372,0.00067952,false


In [30]:
df = simulate(500, 10)
CSV.write("./4.csv", df)
df

Mean CPU: 0.007007174099999999
Mean GPU: 0.0131891974


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.00655469,0.0132898,false
2,0.00533701,0.0130558,false
3,0.00904639,0.0130758,false
4,0.00687869,0.0133207,false
5,0.00640635,0.0130118,false
6,0.0101512,0.0130129,false
7,0.0062136,0.0135257,false
8,0.00538275,0.0131631,false
9,0.00538277,0.013115,false


In [114]:
df = simulate(1000, 10)
CSV.write("./5.csv", df)
df

Mean CPU: 0.03808624000000001
Mean GPU: 0.034511715


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.102373,0.0338499,true
2,0.036042,0.0431727,false
3,0.0331847,0.0337169,false
4,0.0282845,0.0339092,false
5,0.0266976,0.0326739,false
6,0.0309375,0.0339364,false
7,0.0281929,0.0326613,false
8,0.0310332,0.0336127,false
9,0.0331889,0.0349366,false


In [ ]:
df = simulate(5000, 10)
CSV.write("./6.csv", df)
df

Mean CPU: 2.8235273936
Mean GPU: 1.0687475


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,1.72934,1.23179,true
2,1.97266,0.953998,true
3,2.05915,1.18418,true
4,1.86756,0.997821,true
5,2.09291,0.952682,true
6,2.60417,0.986451,true
7,2.50567,1.17317,true
8,3.41478,0.995665,true
9,4.55945,1.19971,true


In [101]:
df = simulate(10000, 10)
CSV.write("./7.csv", df)
df

Total size: 100000000
Mean CPU: 210.2052650053
Mean GPU: 10.711182


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,72.3738,8.33046,true
2,105.515,9.10068,true
3,152.726,9.41615,true
4,188.216,10.1229,true
5,232.592,10.9771,true
6,243.761,10.8583,true
7,264.309,10.9443,true
8,279.165,12.5396,true
9,281.007,11.8832,true


In [18]:
df = simulate(12000, 10)
CSV.write("./8.csv", df)
df

OutOfGPUMemoryError: Out of GPU memory trying to allocate 549.316 MiB
Effective GPU memory usage: 98.41% (1.917 GiB/1.948 GiB)
Memory pool usage: 1.609 GiB (1.844 GiB reserved)
